## Import the neccessary libraries

In [26]:

import math
import random
import operator
import copy
import heapq
import json
from typing import Any
import numpy as np
from collections import defaultdict



## Define the classes in the problem

In [27]:
class Request:
  # """Đại diện cho một khách hàng (request)"""
  """This class represent a customer (request)"""
  def __init__(self, id, location, d_i, r_i, e_i, l_i, ableDrone=None):
    self.id = id
    self.location = location # (x,y)
    self.demand = d_i # the demand of customer i
    self.release_time = r_i # The appearance time of customer i is dynamic
    self.time_window = (e_i, l_i)
    self.ableDrone = ableDrone if ableDrone is not None else 1
    self.is_served = False 


In [28]:
class Vehicle:
  """This class reprent a TRUCK or a DRONE"""
  def __init__(self, id, v_type, capacity, velocity, max_range=None):
    self.id = id
    self.type = v_type # TRUCK or DRONE
    self.capacity = capacity # M_T or M_D
    self.remaining_capacity = capacity
    self.remaining_range = max_range
    # self.max_range = max_range # L_D for DRONE
    self.max_range = max_range if max_range is not None else float('inf')
    # self.rmaining_range = self.max_range
    self.velocity = velocity 
    self.reqQueue = [] # vehicle request queue
    self.current_location = (0,0) # Depot 0
    self.busy_until = 0
    self.routes = [] # list of trips (each trip = list of req ids)
    self.current_trip = None

  def moving_time_to(self, location):
      distance = math.sqrt((self.current_location[0]-location[0])**2+(self.current_location[1]-location[1])**2)
      return distance / self.velocity 

  def moving_time(self, loc_a, loc_b):
      distance = math.sqrt((loc_a[0]-loc_b[0])**2+(loc_a[1]-loc_b[1])**2)
      return distance / self.velocity 

  def sum_of_request_demand(self):
    if not self.reqQueue:
      return 0.0
    return sum([req.demand for req in self.reqQueue])
  
  def distance_to(self, location):
      return math.sqrt((self.current_location[0]-location[0])**2+(self.current_location[1]-location[1])**2)
    
  def median_of_req_loc(self):
    if not self.reqQueue:
      return self.current_location
    avg_x = sum([req.location[0] for req in self.reqQueue])/len(self.reqQueue)
    avg_y = sum([req.location[1] for req in self.reqQueue])/len(self.reqQueue)
    return (avg_x, avg_y)


In [29]:
class Problem:
  """Store the global information of the problem"""
  def __init__(self, depot_time_window_end):
    self.requests = []
    self.vehicles = []
    self.depot_time_window = (0, depot_time_window_end)

  def sum_of_req_demand(self):
    if not self.requests:
      return 0.0
    return sum([req.demand for req in self.requests])


## Define Execution functions of the Terminal set
- The leaf nodes are routing rules and sequencing rules

In [30]:
def routing_rule_terminal(opt, **kwargs): # higher is better (maximize)
  vehicle = kwargs['vehicle']
  p = kwargs['p']
  req = kwargs['req']
  # RT0: càng ít việc càng nên nhận thêm - w=0.05
  if opt == 0:
    total = len(p.requests)
    if total == 0:
      return 0.0
    return 1.0 - len(vehicle.reqQueue) / max(1, len(p.requests))
  # RT1: còn lại càng nhiều chỗ (demand) càng nên nhận thêm - w=0.15
  elif opt == 1:
    sum_demand = p.sum_of_req_demand()
    if sum_demand == 0.0:
      return 0.0
    return (vehicle.capacity - vehicle.sum_of_request_demand()) / sum_demand
  # RT2: càng gần trung tâm hàng đợi thì càng được ưu tiên - w=0.15
  elif opt == 2:
    return -vehicle.moving_time(vehicle.median_of_req_loc(), req.location) / p.depot_time_window[1]
  # RT3: thời gian di chuyển ngắn nhất từ vị trí hiện tại - w=0.5
  elif opt == 3:
    return -vehicle.moving_time_to(req.location)/p.depot_time_window[1]
  # RT4: kích thước request (demand) càng lớn càng được ưu tiên - w=0.05
  elif opt == 4:
    sum_demand = p.sum_of_req_demand()
    if sum_demand == 0.0:
      return 0.0
    return req.demand / sum_demand
  # RT5: ưu tiên DRONE (thường nhân với RT3 để nhận việc gần) - w=0.1
  elif opt == 5:
    return 1 if vehicle.type == "DRONE" else 0.0


In [31]:
def sequencing_rule_terminal(opt, **kwargs):
  vehicle = kwargs['vehicle']
  p = kwargs['p']
  req = kwargs['req']
  # cur_time = kwargs['cur_time']
  cur_time = kwargs.get('cur_time', 0.0)
  # ready_time = kwargs['ready_time']

  # ST0: thời gian di chuyển càng ngắn thì càng được ưu tiên - w=0.4
  if opt == 0:
    return vehicle.moving_time_to(req.location)/p.depot_time_window[1]
  # ST1: càng chờ lâu càng ưu tiên - w=0.05
  elif opt == 1:
    # return (cur_time - ready_time)/p.depot_time_window[1]
    return -(cur_time - req.release_time)/p.depot_time_window[1]
  # ST2: càng gấp càng ưu tiên - w=0.4
  elif opt == 2:
    time_until_close = req.time_window[1] - vehicle.busy_until
    moving_time = vehicle.moving_time_to(req.location)
    if moving_time > time_until_close:
      return float('inf')
    if time_until_close <= 0.001:
      return 1.0
    return moving_time/time_until_close
  # ST3: kích thước request (demand) càng lớn càng được ưu tiên - w=0.05
  elif opt == 3:
    sum_demand = p.sum_of_req_demand()
    if sum_demand == 0.0:
      return 0.0
    return -req.demand/sum_demand
  # ST4: càng trễ càng được ưu tiên - w=0.05
  elif opt == 4:
    return -(cur_time - req.time_window[0])/p.depot_time_window[1]
  # ST5: ưu tiên thời gian xuất hiện sớm - w=0.05
  elif opt == 5:
    return req.release_time / p.depot_time_window[1]

Định nghĩa tập hàm và tập đầu cuối(nút lá)

In [32]:
def protected_div(a,b):
  return a/b if b!=0 else 1.0

## Định nghĩa Nút trong cây GP và Cá thể GPHH (genetic programming hyper heuristic)

In [33]:
FUNC_SET = ['add', 'sub', 'mul', 'div', 'min', 'max']
TERMINAL_ROUTING = [('RT', i) for i in range(6)]  # routing terminal options 0..5
TERMINAL_SEQ = [('ST', i) for i in range(6)]      # sequencing terminal options 0..5
CONST_POOL = [ -2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0 ]

In [34]:
class NodeGP:
  # def __init__(self, op = None, left = None, right = None, terminal = None, const = None, which = 'R'):
  def __init__(self, op = None, left = None, right = None, terminal = None, which = 'R'):
    self.op = op
    self.left = left
    self.right = right
    self.terminal = terminal
    # self.const = const
    self.which = which # 'R' or 'S' - indicates which tree type for terminals

  def is_terminal(self):
    # return (self.terminal is not None) or (self.const is not None)
    return (self.terminal is not None)

  def deepcopy(self):
    return copy.deepcopy(self)

  def size(self):
    if self.is_terminal():
      return 1
    # return 1 + (self.left_size() if self.left else 0) + (self.right_size() if self.right else 0)
    return 1 + (self.left.size() if self.left else 0) + (self.right.size() if self.right else 0)

  def depth(self):
    if self.is_terminal():
      return 1
    return 1 + max(self.left.depth() if self.left else 0, self.right.depth() if self.right else 0)

  def evaluate(self, vehicle, p, req, cur_time=0.0):
    if self.terminal is not None:
      typ, opt = self.terminal
      if typ == 'RT':
        return routing_rule_terminal(opt, vehicle=vehicle, p=p, req=req)
      elif typ == 'ST':
        return sequencing_rule_terminal(opt, vehicle=vehicle, p=p, req=req, cur_time=cur_time)

    # function nodes
    a = self.left.evaluate(vehicle,p,req,cur_time)
    b = self.right.evaluate(vehicle,p,req,cur_time)

    if self.op  == 'add':
      return a + b
    elif self.op =='sub':
      return a-b
    elif self.op == 'mul':
      return a*b
    elif self.op == 'div':
      return protected_div(a,b)
    elif self.op == 'min':
      return min(a,b)
    elif self.op == 'max':
      return max(a,b)

  def to_string(self):
    if self.terminal is not None:
      typ, opt = self.terminal
      return f"{typ}{opt}"
    return f"({self.op} {self.left.to_string()} {self.right.to_string()})"

In [35]:
#class Individual___________________________________
# Individual: pair (R_tree, S_tree)
#___________________________________
class Individual:
  def __init__(self, R_tree:NodeGP, S_tree:NodeGP):
    self.R = R_tree
    self.S = S_tree
    self.fitness = None
    self.f1 = None
    self.f2 = None

  def deepcopy(self):
    return Individual(self.R.deepcopy(), self.S.deepcopy())

  def to_string(self):
    return f"R: {self.R.to_string()}\nS: {self.S.to_string()}"
  
  def evaluate(self, vehicle, p, req, cur_time=0.0):
    r_value = self.R.evaluate(vehicle, p, req, cur_time)
    s_value = self.S.evaluate(vehicle, p, req, cur_time)
    self.fitness = r_value, s_value
    return self.fitness

  def get_fitness(self):
    return self.fitness


## Khởi tạo quần thể

In [36]:
ROUTING_WEIGHTS = [0.05, 0.15, 0.15, 0.50, 0.05, 0.10]  # RT0..RT5
SEQ_WEIGHTS = [0.40, 0.05, 0.40, 0.05, 0.05, 0.05]      # ST0..ST5

In [37]:
def create_greedy_population(pop_size, max_depth=5):
  population = []
  #------------------------------------------------------------------
  #1. hand-crafted strong individuals (dựa trên kinh nghiệm + paper)
  #------------------------------------------------------------------
  strong_individuals =[
      #1. RT3 gần nhất trước, RT1 tránh đầy sức chứa + ST0 thời gian di chuyển ngắn, ST2 ưu tiên gấp
      ("(add RT3 RT1)", "(min ST0 ST2)"),
      #2.RT3 gần nhất trước + ST0 thời gian di chuyển ngắn, ST2 ưu tiên gấp
      ("RT3","(min ST0 ST2)"),
      #3. RT1 tránh đầy sức chứa, RT3 gần nhất trước + ST2 ưu tiên gấp
    #   ("add (mul 2.0 RT1) RT3", "ST2"),
      ("(add RT1 RT3)", "ST2"),
      #4. RT3 gần nhất trước, RT5 ưu tiên DRONE (nhân với R3 là ưu tiên DRONE gần nhất) + ST0 thời gian di chuyển ngắn
      ("(add RT3 (mul RT3 RT5))", "ST0"),
      #5. RT0: càng ít việc càng nên nhận thêm, RT3 gần nhất trước + ST0 thời gian di chuyển ngắn, ST2 ưu tiên gấp
      ("(add RT0 RT3)", "(min ST0 ST2)"),
      #6. RT1 tránh đầy sức chứa, RT2: gần trung tâm hàng đợi  + ST0 thời gian di chuyển ngắn, ST2 ưu tiên gấp
      ("(add RT1 RT2)", "(min ST0 ST2)"),
      #7.
    #   ("RT3", "(div ST2 (add 1.0 (mul 0.1 ST0)))"),
      ("RT3", "(div ST2 ST0)"),
      #8. đơn giản, lựa chọn các luật mạnh nhất là RT3 và ST2
      ("RT3", "ST2"),
  ]
  for r_str, s_str in strong_individuals[:pop_size//3]:
        if len(population) >= pop_size:
            break
        try:
            R_tree = build_tree_from_string(r_str, which='R')
            S_tree = build_tree_from_string(s_str, which='S')
            population.append(Individual(R_tree, S_tree))
        except:
            pass  # nếu lỗi parse thì bỏ qua

  # ------------------------------------------------------------------
  # 2. SEMI-GREEDY: Ưu tiên các terminal mạnh (weighted random)
  # ------------------------------------------------------------------

  while len(population) < pop_size * 2 // 3:
      # Tạo cây với xác suất cao dùng terminal mạnh
      R_tree = make_weighted_random_tree(max_depth, which='R', weights=ROUTING_WEIGHTS)
      S_tree = make_weighted_random_tree(max_depth, which='S', weights=SEQ_WEIGHTS)
      population.append(Individual(R_tree, S_tree))

  # ------------------------------------------------------------------
  # 3. random để giữ đa dạng
  # ------------------------------------------------------------------
  while len(population) < pop_size:
      use_grow = random.random() < 0.5
      R_tree = make_random_tree(max_depth, grow=use_grow, which='R')
      S_tree = make_random_tree(max_depth, grow=use_grow, which='S')
      population.append(Individual(R_tree, S_tree))

  return population[:pop_size]

In [38]:
#------------------------------------
# Random tree generation (full/grow)
#------------------------------------

def random_terminal (which='R'):
  if which == 'R':
    return ('RT', random.randint(0,5))
  else:
    return ('ST', random.randint(0,5))

def make_random_tree(max_depth, grow=False, which='R'):
  if max_depth == 1:
    return NodeGP(terminal=random_terminal(which), which=which)

  if not grow:
    # full initialization: internal nodes until depth limit
    op = random.choice(FUNC_SET)
    left = make_random_tree(max_depth-1, grow=False, which=which)
    right = make_random_tree(max_depth-1, grow=False, which=which)
    return NodeGP(op=op, left=left, right=right, which=which)

  else:
    # grow: choose terminal or function randomly
    if random.random() < 0.5:
      return NodeGP(terminal=random_terminal(which), which=which)
    else:
      op = random.choice(FUNC_SET)
      left = make_random_tree(max_depth-1, grow=True, which=which)
      right = make_random_tree(max_depth-1, grow=True, which=which)
      return NodeGP(op=op, left=left, right=right, which=which)

In [39]:
def weighted_terminal(which='R'):
    if which == 'R':
        opt = random.choices(range(6), weights=ROUTING_WEIGHTS, k=1)[0]
        return ('RT', opt)
    else:
        opt = random.choices(range(6), weights=SEQ_WEIGHTS, k=1)[0]
        return ('ST', opt)

def make_weighted_random_tree(max_depth, grow=True, which='R', weights=None):
    if max_depth == 1 or (grow and random.random() < 0.4):
        return NodeGP(terminal=weighted_terminal(which), which=which)

    op = random.choice(FUNC_SET)
    left = make_weighted_random_tree(max_depth-1, grow=grow, which=which)
    right = make_weighted_random_tree(max_depth-1, grow=grow, which=which)
    return NodeGP(op=op, left=left, right=right, which=which)

In [40]:
def build_tree_from_string(s, which='R'):
    s = s.strip()

    # Terminal đơn
    if s.startswith('RT') and which == 'R':
        num = int(''.join(filter(str.isdigit, s)))
        return NodeGP(terminal=('RT', num), which='R')
    if s.startswith('ST') and which == 'S':
        num = int(''.join(filter(str.isdigit, s)))
        return NodeGP(terminal=('ST', num), which='S')

    # Biểu thức có ngoặc: (op arg1 arg2)
    if s.startswith('(') and s.endswith(')'):
        content = s[1:-1].strip()
        parts = content.split()
        if len(parts) < 1:
            raise ValueError("Invalid expression")

        op = parts[0]
        rest = ' '.join(parts[1:])

        # Tìm 2 phần con
        depth = 0
        split_pos = None
        for i, char in enumerate(rest):
            if char == '(':
                depth += 1
            elif char == ')':
                depth -= 1
            elif char == ' ' and depth == 0 and split_pos is None:
                split_pos = i
                break

        left_str = rest[:split_pos].strip()
        right_str = rest[split_pos:].strip()
        
        left = build_tree_from_string(left_str, which)
        right = build_tree_from_string(right_str, which)

        return NodeGP(op=op, left=left, right=right, which=which)

    raise ValueError(f"Cannot parse: {s}")

In [41]:
# data={
#     "requests": [
#         [
#             1857.8075371321709,
#             -723.5527341657008,
#             0.3772944871169665,
#             1,
#             52.982869225895676,
#             0.0,
#             348.01090081307717
#         ],
#         [
#             3763.891476363072,
#             -3974.1268694620985,
#             29.057640103557844,
#             0,
#             0.0,
#             169.83282674959077,
#             529.8328267495908
#         ],
#         [
#             3598.4433439604213,
#             3517.0689275211994,
#             0.3940784947557429,
#             1,
#             426.1637000755576,
#             319.3540253141671,
#             679.3540253141671
#         ],
#         [
#             -1657.1612805731172,
#             52.171804006016046,
#             0.873772277292186,
#             1,
#             0.0,
#             0.0,
#             232.98286922589568
#         ],
#         [
#             1286.6086590433426,
#             -2762.054795651618,
#             0.15848808715919574,
#             1,
#             168.01090081307717,
#             55.662772444950605,
#             415.66277244495063
#         ],
#         [
#             1568.6879074673716,
#             2456.076308717532,
#             0.593224803960854,
#             1,
#             333.0339355671685,
#             246.16370007555759,
#             606.1637000755576
#         ]
#     ],
#     "truck_vel": 15.6464,
#     "drone_vel": 31.2928,
#     "truck_cap": 400.0,
#     "drone_cap": 2.27,
#     "drone_lim": 700.0,
#     "truck_num": 1,
#     "drone_num": 1,
#     "close": 998.7080506283341
# }

In [42]:
# data={
#     "requests": [
#         [
#             3709.848373093126,
#             -6626.322109716625,
#             0.5237141646387984,
#             1,
#             0.0,
#             62.68043631598857,
#             422.6804363159886
#         ],
#         [
#             -827.2266506062733,
#             3181.863824449649,
#             0.8617803795242891,
#             1,
#             519.9836224333643,
#             668.80285345131,
#             1028.80285345131
#         ],
#         [
#             -4518.310532722009,
#             6765.890855091806,
#             7.729708871578585,
#             0,
#             0.0,
#             339.9836224333643,
#             699.9836224333643
#         ],
#         [
#             6177.911205215619,
#             -974.2045207101374,
#             1.1280061960724481,
#             1,
#             798.4848366972061,
#             698.8853556300646,
#             1058.8853556300646
#         ],
#         [
#             6139.644949987881,
#             -3489.870858944101,
#             0.6730900213514242,
#             1,
#             572.8040504494985,
#             618.4848366972061,
#             978.4848366972061
#         ],
#         [
#             190.6872358052517,
#             -6804.168594973237,
#             0.7745849829511053,
#             1,
#             242.68043631598857,
#             175.28308456670425,
#             535.2830845667042
#         ]
#     ],
#     "truck_vel": 15.6464,
#     "drone_vel": 31.2928,
#     "truck_cap": 400.0,
#     "drone_cap": 2.27,
#     "drone_lim": 700.0,
#     "truck_num": 1,
#     "drone_num": 1,
#     "close": 1757.7707112601292
# }

## Load dữ liệu

In [43]:
data={
    "requests": [
        [
            14510.694403337482,
            16240.750292227222,
            2.672954834762614,
            0,
            0.9217047258718485,
            9217.047258718485,
            9577.047258718485
        ],
        [
            3257.556319472127,
            -28778.660376616477,
            0.22262365012006305,
            1,
            0.7289559082392185,
            7289.559082392185,
            7649.559082392185
        ],
        [
            -26010.08442696178,
            11773.472192855656,
            0.7749532428208182,
            1,
            0.2603162362404881,
            2603.162362404881,
            2963.162362404881
        ],
        [
            -13593.429057649877,
            31186.351050133835,
            1.0654043739394006,
            1,
            0.23112281839511375,
            2311.2281839511375,
            2671.2281839511375
        ],
        [
            2537.357696043769,
            5431.7291049012565,
            0.35180247375615614,
            1,
            0.0670219506813872,
            670.2195068138719,
            1030.219506813872
        ],
        [
            9828.897534736623,
            -19124.18708432133,
            0.8075121533657663,
            1,
            1.1744774958510793,
            11744.774958510792,
            12104.774958510792
        ],
        [
            -14682.599629998225,
            8567.901801280166,
            1.0239417905721824,
            1,
            0.3426476648578368,
            3426.476648578368,
            3786.476648578368
        ],
        [
            7073.738476506514,
            8872.867411950314,
            12.663233438512915,
            0,
            0.9936375637063917,
            9936.375637063917,
            10296.375637063917
        ],
        [
            -29255.118061826935,
            -16206.554762849088,
            0.26194575691124755,
            1,
            1.2920835211462371,
            12920.83521146237,
            13280.83521146237
        ],
        [
            -31136.76064408021,
            29792.951735460283,
            0.08003882124418636,
            1,
            1.1989378869745078,
            11989.378869745076,
            12349.378869745076
        ],
        [
            1291.315870243027,
            9344.489066874421,
            0.5193626898624404,
            1,
            0.012145243173657514,
            121.45243173657514,
            481.45243173657514
        ],
        [
            -2850.555972706583,
            -190.12151332188918,
            0.2536510507756463,
            1,
            0.0919057456901044,
            919.0574569010439,
            1279.057456901044
        ],
        [
            30765.617518186955,
            -10055.626434460504,
            1.0875377625855245,
            1,
            0.2035971271313537,
            2035.9712713135368,
            2395.971271313537
        ],
        [
            -30995.485334346038,
            7129.7011649165415,
            0.970202972771328,
            1,
            0.21677179987952588,
            2167.7179987952586,
            2527.7179987952586
        ],
        [
            -23398.97906512495,
            18824.181667974488,
            0.0932037076003266,
            1,
            1.311775433378584,
            13117.754333785839,
            13477.754333785839
        ],
        [
            25647.403820272466,
            27021.70592710053,
            0.6821859318484339,
            1,
            0.805893556716423,
            8058.93556716423,
            8418.93556716423
        ],
        [
            -28226.772686742093,
            1085.2753196999092,
            0.9379312133370923,
            1,
            0.17391580076741434,
            1739.1580076741434,
            2099.1580076741434
        ],
        [
            28070.70431734434,
            19592.45662998712,
            0.9119317101918545,
            1,
            0.7885930618452858,
            7885.930618452857,
            8245.930618452858
        ],
        [
            -4162.138623526787,
            -2880.313219291682,
            0.7523272936010492,
            1,
            1.4743713922778892,
            14743.71392277889,
            15103.71392277889
        ],
        [
            -759.1307104565365,
            -29768.958481475835,
            1.0634423149232226,
            1,
            0.7025155560674013,
            7025.155560674013,
            7385.155560674013
        ],
        [
            11876.68638950769,
            -7167.2139737152875,
            1.0902640689478797,
            1,
            0.4402584849156363,
            4402.584849156363,
            4762.584849156363
        ],
        [
            -15983.266541459245,
            3735.8269969834755,
            1.1068120847172471,
            1,
            0.37462988747302933,
            3746.2988747302934,
            4106.298874730293
        ],
        [
            14148.22678437538,
            8208.479202340253,
            0.8593356566554905,
            1,
            0.9325104369721574,
            9325.104369721574,
            9685.104369721574
        ],
        [
            27100.716916940615,
            11116.894183936072,
            0.6226877960450552,
            1,
            0.7161298765657628,
            7161.298765657628,
            7521.298765657628
        ],
        [
            -13305.716877484318,
            -8742.252711044906,
            0.9400162757447255,
            1,
            1.1679153747719349,
            11679.153747719349,
            12039.153747719349
        ],
        [
            19826.452607760035,
            -17751.14926205353,
            1.121386439791545,
            1,
            0.3503955923385628,
            3503.955923385628,
            3863.955923385628
        ],
        [
            -17632.02168566228,
            -9537.624386983245,
            0.33436673145942225,
            1,
            1.378341441803755,
            13783.41441803755,
            14143.41441803755
        ],
        [
            19536.813472075937,
            566.1835830188425,
            0.6911061880991004,
            1,
            0.35506603045387375,
            3550.6603045387374,
            3910.6603045387374
        ],
        [
            -31520.006559523994,
            20567.909742788328,
            0.25599247859535473,
            1,
            1.2579482622742175,
            12579.482622742174,
            12939.482622742174
        ],
        [
            -12313.909464999513,
            30811.467562844846,
            25.516254392065882,
            0,
            0.23964431326161223,
            2396.443132616122,
            2756.443132616122
        ],
        [
            2583.4747809260366,
            18277.25818918892,
            1.105010909670599,
            1,
            0.7920434459336785,
            7920.434459336784,
            8280.434459336784
        ],
        [
            -31151.32751928724,
            -31191.812500665957,
            1.046144145449155,
            1,
            1.2016388659439257,
            12016.388659439255,
            12376.388659439255
        ],
        [
            30539.44113976233,
            -21378.209603620562,
            7.287845591183082,
            0,
            0.27810844935902146,
            2781.0844935902146,
            3141.0844935902146
        ],
        [
            10822.809162051924,
            -12291.939613144186,
            0.7687189354727783,
            1,
            1.3906996840844859,
            13906.996840844859,
            14266.996840844859
        ],
        [
            31317.17834743516,
            -7361.439481529208,
            0.9693988645972885,
            1,
            0.2211734765923483,
            2211.734765923483,
            2571.734765923483
        ],
        [
            -21031.8081505654,
            8157.865557846512,
            0.012596240690029676,
            1,
            0.2996397257429704,
            2996.3972574297036,
            3356.3972574297036
        ],
        [
            25979.053385693405,
            -12550.713240259525,
            14.769380858952005,
            0,
            0.16909821406436892,
            1690.9821406436893,
            2050.9821406436895
        ],
        [
            13371.931225739156,
            -31637.55778716877,
            0.48824419532970853,
            1,
            1.2583202752254803,
            12583.202752254801,
            12943.202752254801
        ],
        [
            -618.5606662840346,
            15907.721091760819,
            19.922259470338144,
            0,
            0.388830534745155,
            3888.3053474515496,
            4248.305347451549
        ],
        [
            11707.676003954633,
            11170.033852892915,
            1.7994194232680776,
            1,
            0.960581562860098,
            9605.81562860098,
            9965.81562860098
        ],
        [
            -7662.418817604266,
            27867.456870790673,
            0.692139695647081,
            1,
            0.27890747515518466,
            2789.0747515518465,
            3149.0747515518465
        ],
        [
            12920.45077613332,
            -21106.086651154033,
            0.1285319383719481,
            1,
            1.332758249058716,
            13327.58249058716,
            13687.58249058716
        ],
        [
            -5654.325949089917,
            2423.582789695468,
            0.12733151459848407,
            1,
            0.001658968302495231,
            16.589683024952308,
            376.5896830249523
        ],
        [
            25535.94493916875,
            4500.789047732284,
            29.362904818186564,
            0,
            0.3055125378420833,
            3055.125378420833,
            3415.125378420833
        ],
        [
            -8811.577298201884,
            -15270.54964780402,
            1.1204306215424327,
            1,
            1.3786421060668803,
            13786.421060668803,
            14146.421060668803
        ],
        [
            -15184.758622027279,
            12726.775051828517,
            1.1263899693813615,
            1,
            1.3771575372483276,
            13771.575372483276,
            14131.575372483276
        ],
        [
            22814.918876367938,
            -26675.240053634025,
            12.163117391394595,
            0,
            0.20633994209979464,
            2063.3994209979464,
            2423.3994209979464
        ],
        [
            24080.522986642172,
            16311.810243299653,
            0.7630630119574304,
            1,
            0.7545352191547765,
            7545.352191547765,
            7905.352191547765
        ],
        [
            -5738.077866621482,
            6246.21208148901,
            0.3535401809480772,
            1,
            0.05973794552983642,
            597.3794552983642,
            957.3794552983642
        ],
        [
            -25841.270119293426,
            18704.103913859875,
            0.7982245326316104,
            1,
            1.2961472943457417,
            12961.472943457416,
            13321.472943457416
        ],
        [
            28442.575436991756,
            -26571.434204011883,
            0.12586897402061162,
            1,
            0.24231380102580521,
            2423.138010258052,
            2783.138010258052
        ],
        [
            -17830.88358767739,
            24393.107440817545,
            0.3502620734006296,
            1,
            0.17511340498518366,
            1751.1340498518364,
            2111.1340498518366
        ],
        [
            -22978.35066950899,
            -13530.673660116754,
            0.9427441636012241,
            1,
            1.3356932535872112,
            13356.932535872113,
            13716.932535872113
        ],
        [
            -27309.927312044863,
            30386.97691910629,
            1.0722191313731173,
            1,
            1.174186742480587,
            11741.867424805869,
            12101.867424805869
        ],
        [
            4743.33491048603,
            28594.895985938463,
            22.04332894308262,
            0,
            0.7246715078870304,
            7246.715078870304,
            7606.715078870304
        ],
        [
            3465.1736094110665,
            17191.032425748304,
            0.4388700251556369,
            1,
            0.8009849785556339,
            8009.849785556338,
            8369.849785556338
        ],
        [
            -5612.922960003192,
            5582.63741133319,
            0.45942536366087977,
            1,
            0.06189586712107447,
            618.9586712107447,
            978.9586712107447
        ],
        [
            -21019.795689717448,
            -26262.828367958777,
            0.2861169810813159,
            1,
            1.273648341039281,
            12736.483410392808,
            13096.483410392808
        ],
        [
            -22601.308483449746,
            -17358.93136422195,
            0.48611369014529326,
            1,
            1.2489243111506925,
            12489.243111506923,
            12849.243111506923
        ],
        [
            -30598.869715506426,
            6807.553511949363,
            0.37948370870964004,
            1,
            0.21350611016945842,
            2135.061101694584,
            2495.061101694584
        ],
        [
            13295.026458337434,
            22102.223130426744,
            0.02238321581252956,
            1,
            0.6560477829189947,
            6560.4778291899465,
            6920.4778291899465
        ],
        [
            16633.172110282347,
            22226.908423128614,
            10.802736210991254,
            0,
            0.8762970789623733,
            8762.970789623732,
            9122.970789623732
        ],
        [
            16989.035125978997,
            18477.63005505647,
            13.012238105176221,
            0,
            0.9003673371911893,
            9003.673371911893,
            9363.673371911893
        ],
        [
            -931.8637204107273,
            26157.582155513963,
            24.350781480961395,
            0,
            0.3232905495404899,
            3232.905495404899,
            3592.905495404899
        ],
        [
            -15098.091547570768,
            7896.322817971693,
            0.2849191321346278,
            1,
            0.3376003931599803,
            3376.003931599803,
            3736.003931599803
        ],
        [
            13933.363475044936,
            -13403.931214963253,
            0.4908304806547534,
            1,
            0.39828664640886957,
            3982.8664640886955,
            4342.8664640886955
        ],
        [
            -7528.865981054281,
            28817.521803310305,
            0.7774828684647337,
            1,
            0.2727756753241218,
            2727.7567532412177,
            3087.7567532412177
        ],
        [
            14161.869305942157,
            13723.962469573436,
            0.20389509890470608,
            1,
            0.9379439009831174,
            9379.439009831174,
            9739.439009831174
        ],
        [
            -4507.256341344836,
            -21150.365527482038,
            0.19042768335665133,
            1,
            0.6424485175141396,
            6424.485175141395,
            6784.485175141395
        ],
        [
            27740.60575155826,
            24519.655686313377,
            0.42557832932578454,
            1,
            0.7184658627654132,
            7184.658627654132,
            7544.658627654132
        ],
        [
            -1181.8032388060649,
            -7025.809612951416,
            0.13587557878479187,
            1,
            0.10996122658798309,
            1099.6122658798308,
            1459.6122658798308
        ],
        [
            -13862.364890063858,
            4338.523160440349,
            0.27407010925029335,
            1,
            0.3887217766482585,
            3887.217766482585,
            4247.217766482585
        ],
        [
            13294.972491229724,
            17732.361553367045,
            0.1974637681384769,
            1,
            0.6281189201940679,
            6281.189201940679,
            6641.189201940679
        ],
        [
            6963.1566910076435,
            -1318.8690251670134,
            0.5884419051336783,
            1,
            0.009986104711149824,
            99.86104711149824,
            459.86104711149824
        ],
        [
            10416.521351687878,
            -24283.71867927397,
            0.34418006349506947,
            1,
            1.2076665124353223,
            12076.665124353223,
            12436.665124353223
        ],
        [
            -3468.8512046945834,
            14439.986609264732,
            0.7844644958130039,
            1,
            0.40932083595917285,
            4093.2083595917284,
            4453.208359591728
        ],
        [
            -28506.558849063076,
            -407.17289896554803,
            0.33545540707227955,
            1,
            0.1642110301666589,
            1642.1103016665888,
            2002.1103016665888
        ],
        [
            29025.785881595242,
            26008.796737187964,
            0.446854528470267,
            1,
            0.7310376722297509,
            7310.376722297508,
            7670.376722297508
        ],
        [
            15503.929243618937,
            -26513.52801226225,
            10.013969726527531,
            0,
            1.2940510540143488,
            12940.510540143487,
            13300.510540143487
        ],
        [
            13884.598740502577,
            -24136.469217382702,
            0.9492755783731273,
            1,
            1.3124336832786512,
            13124.336832786512,
            13484.336832786512
        ],
        [
            26512.579461724188,
            17464.447041613224,
            0.46193269368194323,
            1,
            0.7717364257782054,
            7717.364257782054,
            8077.364257782054
        ],
        [
            11542.121749982532,
            -2813.9510883550843,
            0.40895350142339126,
            1,
            0.41054131642929537,
            4105.413164292953,
            4465.413164292953
        ],
        [
            20100.77896294042,
            26412.871599056612,
            0.5831218583022166,
            1,
            0.8415563267748445,
            8415.563267748445,
            8775.563267748445
        ],
        [
            -1172.8006537068043,
            7843.222677206654,
            0.5138216162632187,
            1,
            0.021365970908164663,
            213.6597090816466,
            573.6597090816466
        ],
        [
            -6225.0404961237255,
            -4392.551233961572,
            1.0203023701449914,
            1,
            0.023516984340906415,
            235.16984340906413,
            595.1698434090641
        ],
        [
            25729.303939873276,
            29830.27773476926,
            0.9523659020634255,
            1,
            0.7879356514523459,
            7879.356514523459,
            8239.356514523459
        ],
        [
            17993.97323343059,
            -15436.212230107549,
            9.464133842735928,
            0,
            0.36926537746654303,
            3692.6537746654303,
            4052.6537746654303
        ],
        [
            -16744.572662250506,
            29436.50171579928,
            0.5510753347521953,
            1,
            0.20808622764324147,
            2080.8622764324145,
            2440.8622764324145
        ],
        [
            -11295.359680403268,
            3064.972874287592,
            0.32923842483107896,
            1,
            0.4070362904506762,
            4070.362904506762,
            4430.362904506762
        ],
        [
            20653.307014667822,
            20692.84044161477,
            0.6943177252335692,
            1,
            0.8365183035401026,
            8365.183035401025,
            8725.183035401025
        ],
        [
            23334.340903077114,
            -9389.767624006328,
            0.7014794130711728,
            1,
            0.14275721939310052,
            1427.5721939310051,
            1787.5721939310051
        ],
        [
            27052.53922253942,
            32029.527962583812,
            0.08107615700759756,
            1,
            0.771531607110927,
            7715.3160711092705,
            8075.3160711092705
        ],
        [
            -12287.505625589185,
            12248.31156130952,
            0.310478297665388,
            1,
            1.3959254000773624,
            13959.254000773624,
            14319.254000773624
        ],
        [
            20469.35227091287,
            2780.4817588228666,
            21.411114439421972,
            0,
            0.33971007799258085,
            3397.100779925808,
            3757.100779925808
        ],
        [
            4913.407580782822,
            -27004.186787201594,
            0.7935636133224007,
            1,
            0.7444678203272247,
            7444.678203272247,
            7804.678203272247
        ],
        [
            20160.577099821694,
            14132.585085944958,
            0.4200105924634065,
            1,
            0.878564610654678,
            8785.64610654678,
            9145.64610654678
        ],
        [
            15063.92200945871,
            -28405.07654680067,
            0.8103753269776262,
            1,
            1.2816389270360256,
            12816.389270360256,
            13176.389270360256
        ],
        [
            -10012.594735556551,
            -12260.580063402855,
            0.2380168613144592,
            1,
            1.399354438944099,
            13993.54438944099,
            14353.54438944099
        ],
        [
            11800.75258528957,
            -15738.437943496096,
            0.5615951361830556,
            1,
            1.3678026706183108,
            13678.026706183107,
            14038.026706183107
        ],
        [
            5927.656584677681,
            -3106.0537778009425,
            0.8162511448922212,
            1,
            0.003385545178125091,
            33.855451781250906,
            393.8554517812509
        ]
    ],
    "truck_vel": 15.6464,
    "drone_vel": 31.2928,
    "truck_cap": 400.0,
    "drone_cap": 2.27,
    "drone_lim": 700.0,
    "truck_num": 4,
    "drone_num": 4,
    "close": 29847.42784555778
}

In [44]:
def load_problem(data) -> Problem:
    L_w = float(data["close"])
    prob = Problem(depot_time_window_end=L_w)

    for idx, r in enumerate(data["requests"]):
        req = Request(
            id=idx,
            location=(float(r[0]), float(r[1])),
            d_i=float(r[2]),
            r_i=float(r[4]),
            e_i=float(r[5]),
            l_i=float(r[6]),
            ableDrone=int(r[3])
        )
        prob.requests.append(req)

    truck_cap = float(data["truck_cap"])
    drone_cap = float(data["drone_cap"])
    drone_lim = float(data["drone_lim"])
    truck_vel = float(data["truck_vel"])
    drone_vel = float(data["drone_vel"])
    truck_cnt = int(data["truck_num"])
    drone_cnt = int(data["drone_num"])

    vid = 0
    for i in range(truck_cnt):
        truck = Vehicle(
            id=vid,
            v_type="TRUCK",
            capacity=truck_cap,
            velocity=truck_vel, 
            max_range=None
        )
        prob.vehicles.append(truck)
        vid += 1

    for i in range(drone_cnt):
        drone = Vehicle(
            id=vid,
            v_type="DRONE",
            capacity=drone_cap,
            velocity=drone_vel,
            max_range=drone_lim
        )
        prob.vehicles.append(drone)
        vid += 1

    return prob

## Giả lập phân bổ sự kiện 

In [ ]:
def simulate_policy(individual: Individual, problem: Problem) -> dict:
    local_p = copy.deepcopy(problem)
    close_time = local_p.depot_time_window[1]

    event_queue = []
    for req in local_p.requests:
        heapq.heappush(event_queue, (req.release_time, "ARRIVE", req.id))
    heapq.heappush(event_queue, (close_time + 1e-9, "END", None))

    cur_time = 0.0

    while event_queue:
        time, ev_type, payload = heapq.heappop(event_queue)
        cur_time = time
        if ev_type == "END":
            break

        if ev_type == "ARRIVE":
            req = next((r for r in local_p.requests if r.id == payload), None)
            if not req or req.is_served or cur_time > req.time_window[1] + 1e-6:
                continue

            # compute routing scores and candidate vehicles
            # tính toán điểm routing và các vehicle ứng cử viên
            candidates = []
            for veh in local_p.vehicles:
                if veh.type == "DRONE" and req.ableDrone == 0:
                    continue
                # capacity: đảm bảo tổng demand trong hàng đợi + demand mới này <= capacity (một vehicle không thể nhận tổng demand vượt quá capacity)
                if veh.sum_of_request_demand() + req.demand > veh.capacity + 1e-6:
                    continue

                # range feasibility for drone (use distances)
                # kiểm tra phạm vi cho drone (sử dụng khoảng cách)
                if veh.type == "DRONE":
                    d_moving = math.sqrt((veh.current_location[0] - req.location[0])**2 +
                                         (veh.current_location[1] - req.location[1])**2)
                    d_returning = math.sqrt(req.location[0]**2 + req.location[1]**2)
                    total_time = d_moving / veh.velocity + d_returning / veh.velocity
                    # if veh.remaining_range < d_moving + d_returning - 1e-6:
                    #     continue
                    if veh.remaining_range < total_time - 1e-6:
                        continue

                score = individual.R.evaluate(veh, local_p, req, cur_time)
                candidates.append((score, veh))

            if not candidates:
                continue

            candidates.sort(key=lambda x: x[0], reverse=True)
            for score, veh in candidates:
                start_service_time = max(cur_time, veh.busy_until)
                travel_time = veh.moving_time_to(req.location)
                arrival_time = start_service_time + travel_time
                if arrival_time > req.time_window[1] + 1e-6:
                    continue
                veh.reqQueue.append(req)
                # if vehicle is idle now (busy_until <= cur_time), dispatch one step immediately (and schedule VEH_FREE via event_queue)
                # nếu vehicle đang rảnh bây giờ (busy_until <= cur_time) ta sẽ điều phối ngay một bước (và lên lịch VEH_FREE thông qua event_queue)
                if veh.busy_until <= cur_time + 1e-6:
                    _dispatch_vehicle(veh, individual, local_p, cur_time, event_queue)
                break

        elif ev_type == "VEH_FREE":
            # vehicle becomes free at this time: dispatch next step if it has queued requests
            # vehicle trở nên rảnh vào thời điểm này: điều phối bước tiếp theo nếu nó có các request trong hàng đợi
            vid = payload
            veh = next((v for v in local_p.vehicles if v.id == vid), None)
            if veh and veh.reqQueue:
                _dispatch_vehicle(veh, individual, local_p, cur_time, event_queue)

        # After handling event, also check any vehicles whose busy_until == cur_time (safety)
        # Sau khi xử lý sự kiện, cũng kiểm tra bất kỳ vehicle nào có busy_until == cur_time (để an toàn)
        for veh in local_p.vehicles:
            if abs(veh.busy_until - cur_time) < 1e-6 and veh.reqQueue:
                _dispatch_vehicle(veh, individual, local_p, cur_time, event_queue)

    # compute results
    # tính toán kết quả
    served_count = sum(1 for r in local_p.requests if r.is_served)
    total = len(local_p.requests)
    makespan = max((v.busy_until for v in local_p.vehicles), default=0.0)

    f1 = served_count / total if total > 0 else 0.0
    f2 = max(0.0, 1.0 - makespan / close_time)

    individual.f1 = f1
    individual.f2 = f2
    individual.fitness = (f1, f2)

    return {
        "served": served_count,
        "total": total,
        "makespan": makespan,
        "unserved": total - served_count,
        "ratio": f1,
        "f1": f1,
        "f2": f2
    }


def _dispatch_vehicle(veh: Vehicle, ind: Individual, p: Problem, start_time: float, event_queue):
    """
    Dispatch (Điều phối) là một hành động cho veh vào thời gian start_time:
      - chọn req khả thi tốt nhất từ veh.reqQueue sử dụng quy tắc sắp xếp S
      - HOẶC, nếu không khả thi, veh phải trở về depot
    Sau khi lên lịch hành động, đặt veh.busy_until và đẩy (veh.busy_until, "VEH_FREE", veh.id) vào event_queue.
    """
    ready_time_for_move = max(start_time, veh.busy_until)

    candidates = []
    for req in list(veh.reqQueue): 
        if req.is_served:
            continue

        if veh.type == "DRONE":
            d_moving = veh.distance_to(req.location)
            d_returning = math.sqrt(req.location[0]**2 + req.location[1]**2)
            total_time = d_moving / veh.velocity + d_returning / veh.velocity
            if veh.remaining_range < total_time - 1e-6:
                continue

        travel_time = veh.moving_time_to(req.location)
        arrival = ready_time_for_move + travel_time
        if arrival > req.time_window[1] + 1e-6:
            continue

        score = ind.S.evaluate(veh, p, req, start_time)
        candidates.append((score, req, travel_time))

    if candidates:
        best = min(candidates, key=lambda x: x[0])
        _, next_req, travel_time = best

        arrival = ready_time_for_move + travel_time
        service_start = max(arrival, next_req.time_window[0])  # service time = 0

        # update vehicle state
        # cập nhật trạng thái vehicle
        veh.remaining_capacity -= next_req.demand
        travel_distance = veh.distance_to(next_req.location)
        if veh.type == "DRONE":
            veh.remaining_range -= travel_distance/veh.velocity

        # tạo trip mới nếu rời khỏi depot
        if veh.current_location == (0, 0):
            # # start new trip
            # if not hasattr(veh, "routes"):
            #     veh.routes = []
            veh.routes.append([])  # new trip
        # if hasattr(veh, "routes") and veh.routes:
            veh.routes[-1].append(next_req.id)

        veh.current_location = next_req.location
        veh.busy_until = service_start
        next_req.is_served = True

        veh.reqQueue = [r for r in veh.reqQueue if r.id != next_req.id]

        heapq.heappush(event_queue, (veh.busy_until, "VEH_FREE", veh.id))

    else:
        # no feasible task: go back to depot (if not already there)
        # nếu không có task nào khả thi: quay về depot (nếu chưa ở đó)
        if veh.current_location == (0, 0):
            return # vehicle đã ở depot, không làm gì
        back_time = veh.moving_time(veh.current_location, (0, 0))
        veh.busy_until = ready_time_for_move + back_time
        veh.current_location = (0, 0)
        # reset capacity/range
        veh.remaining_capacity = veh.capacity
        if veh.type == "DRONE":
            veh.remaining_range = veh.max_range

        heapq.heappush(event_queue, (veh.busy_until, "VEH_FREE", veh.id))

In [46]:
# Khởi tạo quần thể tham lam
population = create_greedy_population(pop_size=300, max_depth=4)
print(len(population))
print(f"Khởi tạo xong {len(population)} cá thể thông minh!")
for i, ind in enumerate(population):
    print(f"Ind {i}: R = {ind.R.to_string()} | S = {ind.S.to_string()}")

300
Khởi tạo xong 300 cá thể thông minh!
Ind 0: R = (add RT3 RT1) | S = (min ST0 ST2)
Ind 1: R = RT3 | S = (min ST0 ST2)
Ind 2: R = (add RT1 RT3) | S = ST2
Ind 3: R = (add RT3 (mul RT3 RT5)) | S = ST0
Ind 4: R = (add RT0 RT3) | S = (min ST0 ST2)
Ind 5: R = (add RT1 RT2) | S = (min ST0 ST2)
Ind 6: R = RT3 | S = (div ST2 ST0)
Ind 7: R = RT3 | S = ST2
Ind 8: R = (min (max (div RT3 RT3) (max RT3 RT3)) RT2) | S = ST2
Ind 9: R = RT3 | S = (max ST0 ST0)
Ind 10: R = (max RT3 RT3) | S = ST5
Ind 11: R = RT3 | S = (add ST0 ST2)
Ind 12: R = (min (div (sub RT0 RT3) RT3) RT1) | S = ST5
Ind 13: R = (add (max (div RT3 RT1) RT1) RT3) | S = (min (sub (sub ST0 ST2) ST2) (min ST0 (mul ST3 ST2)))
Ind 14: R = RT3 | S = (mul (mul (div ST2 ST5) (max ST0 ST3)) (div (min ST5 ST2) (div ST3 ST0)))
Ind 15: R = (max RT3 (div (div RT1 RT3) (div RT0 RT3))) | S = (max (min (min ST0 ST3) ST2) (add (mul ST0 ST2) (add ST2 ST2)))
Ind 16: R = (div RT2 RT3) | S = (max (min ST2 ST0) ST0)
Ind 17: R = RT0 | S = ST2
Ind 18: R =

## Các thao tác trên cây

In [ ]:
# ---------------------------------------------
# 0. Helper Functions cho Thao tác Cây
# ---------------------------------------------
def count_nodes(node):
    if node is None: return 0
    return 1 + count_nodes(node.left) + count_nodes(node.right)

def get_node_at_index(node, target_idx, current_idx=0):
    """Trả về node tại chỉ số index (theo duyệt pre-order) và index tiếp theo"""
    if current_idx == target_idx:
        return node, current_idx + 1
    
    current_idx += 1
    if node.left:
        found, new_idx = get_node_at_index(node.left, target_idx, current_idx)
        if found: return found, new_idx
        current_idx = new_idx
    
    if node.right:
        found, new_idx = get_node_at_index(node.right, target_idx, current_idx)
        if found: return found, new_idx
        current_idx = new_idx
        
    return None, current_idx

def replace_node_at_index(root, target_idx, new_subtree, current_idx=0):
    """Tạo ra một bản sao của cây với node tại target_idx được thay thế"""
    if current_idx == target_idx:
        return new_subtree.deepcopy(), current_idx + 1 
    
    new_node = NodeGP(op=root.op, terminal=root.terminal, which=root.which)
    
    current_idx += 1
    
    if root.left:
        if current_idx <= target_idx: 
             size_left = count_nodes(root.left)
             if target_idx < current_idx + size_left:
                 new_left, new_idx = replace_node_at_index(root.left, target_idx, new_subtree, current_idx)
                 new_node.left = new_left
                 new_node.right = root.right.deepcopy() if root.right else None
                 return new_node, new_idx
             else:
                 new_node.left = root.left.deepcopy()
                 current_idx += size_left
    
    if root.right:
         new_right, new_idx = replace_node_at_index(root.right, target_idx, new_subtree, current_idx)
         new_node.right = new_right
         return new_node, new_idx
         
    return new_node, current_idx

# ---------------------------------------------------
# 1. Các Toán tử Di truyền 
# ---------------------------------------------------
def gp_crossover(parent1, parent2, max_depth=6):
    """Lai ghép Subtree giữa 2 cá thể"""
    child1 = parent1.deepcopy()
    child2 = parent2.deepcopy()
    
    # Chọn lai cây R hay cây S
    if random.random() < 0.5: 
        tree1 = child1.R
        tree2 = child2.R
        which = 'R'
    else: 
        tree1 = child1.S
        tree2 = child2.S
        which = 'S'
        
    # Đếm số node
    size1 = count_nodes(tree1)
    size2 = count_nodes(tree2)
    
    # Chọn điểm cắt ngẫu nhiên
    idx1 = random.randint(0, size1 - 1)
    idx2 = random.randint(0, size2 - 1)
    
    # Lấy subtree tại điểm cắt
    subtree1, _ = get_node_at_index(tree1, idx1)
    subtree2, _ = get_node_at_index(tree2, idx2)
    
    # Kiểm tra depth limit
    if (tree1.depth() - subtree1.depth() + subtree2.depth() <= max_depth) and \
       (tree2.depth() - subtree2.depth() + subtree1.depth() <= max_depth):
        
        # Thực hiện hoán đổi bằng cách xây dựng lại cây
        if which == 'R':
            child1.R, _ = replace_node_at_index(child1.R, idx1, subtree2)
            child2.R, _ = replace_node_at_index(child2.R, idx2, subtree1)
        else:
            child1.S, _ = replace_node_at_index(child1.S, idx1, subtree2)
            child2.S, _ = replace_node_at_index(child2.S, idx2, subtree1)
            
    return child1, child2


def gp_mutation(individual, max_depth=6):
    """Đột biến Subtree có kiểm soát độ sâu"""
    child = individual.deepcopy()
    
    if random.random() < 0.5:
        target_tree = child.R
        which = 'R'
    else:
        target_tree = child.S
        which = 'S'
        
    size = count_nodes(target_tree)

    for _ in range(10):
        idx = random.randint(0, size - 1)
        
        # Tạo cây con đột biến (độ sâu nhỏ)
        mutation_subtree = make_random_tree(max_depth=random.randint(1, 3), grow=True, which=which)
        new_tree, _ = replace_node_at_index(target_tree, idx, mutation_subtree)
        
        if new_tree.depth() <= max_depth:
            if which == 'R':
                child.R = new_tree
            else:
                child.S = new_tree
            return child
            
    # Nếu thử nhiều lần mà vẫn vi phạm độ sâu, trả về cá thể gốc (không đột biến)
    return individual.deepcopy()

### Thuật toán sắp xếp nhanh không trội NSGA-II giải quyết đa mục tiêu

In [48]:
# ---------------------------------------------
# 2. NSGA-II Components
# ---------------------------------------------
def dominate(ind1, ind2):
    """
    Trả về True nếu ind1 dominate ind2.
    Mục tiêu: Maximize F1 (Tỉ lệ request được phục vụ), Maximize F2 (1 - Makespan / L_w   --Makespan là thời gian hoàn thành nhiệm vụ)
    """
    f1_a, f2_a = ind1.fitness
    f1_b, f2_b = ind2.fitness
    
    # Dominate nếu tốt hơn hoặc bằng ở mọi mục tiêu và tốt hơn ít nhất 1 mục tiêu
    if (f1_a >= f1_b and f2_a >= f2_b) and (f1_a > f1_b or f2_a > f2_b):
        return True
    return False

def fast_non_dominated_sorting(population):
    fronts = [[]]
    domination_count = defaultdict(int)
    dominated_solutions = defaultdict(list)
    
    for p in population:
        for q in population:
            if p == q: continue
            if dominate(p, q):
                dominated_solutions[p].append(q)
            elif dominate(q, p):
                domination_count[p] += 1
        
        if domination_count[p] == 0:
            fronts[0].append(p)
            
    i = 0
    while fronts[i]:
        next_front = []
        for p in fronts[i]:
            for q in dominated_solutions[p]:
                domination_count[q] -= 1
                if domination_count[q] == 0:
                    next_front.append(q)
        i += 1
        if next_front:
            fronts.append(next_front)
        else:
            break
            
    return fronts

def crowding_distance_assignment(front):
    l = len(front)
    if l == 0: return
    
    for ind in front:
        ind.distance = 0
        
    # Tính cho từng mục tiêu (2 mục tiêu)
    for m in range(2):
        # Sắp xếp theo mục tiêu m
        front.sort(key=lambda x: x.fitness[m])
        
        front[0].distance = float('inf')
        front[-1].distance = float('inf')
        
        f_min = front[0].fitness[m]
        f_max = front[-1].fitness[m]
        
        if f_max == f_min: continue
        
        for i in range(1, l-1):
            front[i].distance += (front[i+1].fitness[m] - front[i-1].fitness[m]) / (f_max - f_min)

def nsga2_select(population, k):
    """Chọn k cá thể tốt nhất cho thế hệ sau"""
    fronts = fast_non_dominated_sorting(population)
    new_pop = []
    
    for front in fronts:
        crowding_distance_assignment(front)
        front.sort(key=lambda x: x.distance, reverse=True)
        
        if len(new_pop) + len(front) <= k:
            new_pop.extend(front)
        else:
            needed = k - len(new_pop)
            new_pop.extend(front[:needed])
            break
            
    return new_pop

## Giải thuật tiến hóa

In [49]:
# ---------------------------------------------------
# Helper: gán rank và distance cho toàn bộ quần thể
# ---------------------------------------------------
def assign_rank_and_distance(population):
    """
    Tính toán và gán thuộc tính .rank và .distance cho mọi cá thể 
    để phục vụ cho Tournament Selection.
    """
    fronts = fast_non_dominated_sorting(population)
    for r, front in enumerate(fronts):
        crowding_distance_assignment(front)
        for ind in front:
            ind.rank = r 

# ---------------------------------------------------
# Helper: Tournament Selection (NSGA-II Style)
# ---------------------------------------------------
def nsga2_tournament_selection(population, k=2):
    """
    Chọn k cá thể ngẫu nhiên, trả về cá thể tốt nhất dựa trên:
    1. Rank thấp hơn (tốt hơn).
    2. Nếu Rank bằng nhau, Crowding Distance lớn hơn (tốt hơn).
    """
    candidates = random.sample(population, k)
    
    best = candidates[0]
    for cand in candidates[1:]:
        if cand.rank < best.rank:
            best = cand
        elif cand.rank == best.rank and cand.distance > best.distance:
            best = cand
            
    return best

# -------------------------------------
# 3. Main Evolution Loop
# ------------------------------------
def gphh_evolution(problem, pop_size=50, max_gen=20, cx_pb=0.8, mut_pb=0.2, max_depth=6, tourn_size=4):
    
    # 1. Khởi tạo quần thể
    population = create_greedy_population(pop_size, max_depth=4)
    
    # 2. Đánh giá quần thể ban đầu
    for ind in population:
        simulate_policy(ind, problem) 
        
    # gán rank/distance ban đầu để chạy tournament_selection lần đầu tiên
    assign_rank_and_distance(population)

    # Lưu best history
    stats_history = []
    
    for gen in range(1, max_gen + 1):
        offspring = []

        while len(offspring) < pop_size:
            p1 = nsga2_tournament_selection(population, k=tourn_size)
            p2 = nsga2_tournament_selection(population, k=tourn_size)
            
            if random.random() < cx_pb:
                c1, c2 = gp_crossover(p1, p2, max_depth)
                
                if random.random() < mut_pb: c1 = gp_mutation(c1, max_depth)
                if random.random() < mut_pb: c2 = gp_mutation(c2, max_depth)
                
                # Reset fitness cho con mới sinh ra
                c1.fitness = None
                c2.fitness = None
                offspring.extend([c1, c2])
            else:
                c1 = p1.deepcopy()
                if random.random() < mut_pb:
                    c1 = gp_mutation(c1, max_depth)
                c1.fitness = None
                offspring.append(c1)
        
        # Cắt bớt nếu dư
        offspring = offspring[:pop_size]
        
        # Đánh giá offspring
        for ind in offspring:
            simulate_policy(ind, problem)
            
        # Gộp quần thể (Cha mẹ + Con cái)
        combined_pop = population + offspring
        
        # Chọn lọc sinh tồn (NSGA-II Selection cho thế hệ sau)
        population = nsga2_select(combined_pop, pop_size)
        
        # Cập nhật rank/distance cho quần thể mới
        assign_rank_and_distance(population)
        
        # Thống kê
        best_f1 = max(population, key=lambda x: x.f1)
        best_f2 = max(population, key=lambda x: x.f2)
        
        stats = {
            "gen": gen,
            "best_served_ratio": best_f1.f1,
            "best_makespan_score": best_f2.f2
        }
        stats_history.append(stats)
        
        print(f"Gen {gen:3d} | Best Ratio: {best_f1.f1:.3f} | Best Time Score: {best_f2.f2:.3f}")
        
    # Trả về quần thể cuối cùng và Pareto Front
    fronts = fast_non_dominated_sorting(population)
    pareto_front = fronts[0]
    
    return population, pareto_front, stats_history

In [50]:
# --- CHẠY THỬ NGHIỆM ---

# 1. Load Problem
problem_instance = load_problem(data)
print(f"Problem Loaded: {len(problem_instance.requests)} requests, {len(problem_instance.vehicles)} vehicles")

# 2. Chạy thuật toán GPHH
POP_SIZE = 200
GENS = 30

final_pop, pareto, history = gphh_evolution(
    problem_instance, 
    pop_size=POP_SIZE, 
    max_gen=GENS, 
    max_depth=5
)

print("\n=== FINAL PARETO FRONT ===")
print(f"Found {len(pareto)} non-dominated solutions.")

for i, ind in enumerate(pareto):
    close_time = problem_instance.depot_time_window[1]
    raw_makespan = (1.0 - ind.f2) * close_time
    
    print(f"\nSolution {i+1}:")
    print(f"  Served Ratio: {ind.f1:.2%} ({int(ind.f1*len(problem_instance.requests))} reqs)")
    print(f"  Makespan:     {raw_makespan:.1f}")
    print(f"  Rule R: {ind.R.to_string()}")
    print(f"  Rule S: {ind.S.to_string()}")

Problem Loaded: 100 requests, 8 vehicles


Gen   1 | Best Ratio: 0.840 | Best Time Score: 0.499
Gen   2 | Best Ratio: 0.860 | Best Time Score: 0.499
Gen   3 | Best Ratio: 0.860 | Best Time Score: 0.499
Gen   4 | Best Ratio: 0.860 | Best Time Score: 0.499
Gen   5 | Best Ratio: 0.860 | Best Time Score: 0.499
Gen   6 | Best Ratio: 0.860 | Best Time Score: 0.499
Gen   7 | Best Ratio: 0.860 | Best Time Score: 0.499
Gen   8 | Best Ratio: 0.870 | Best Time Score: 0.499
Gen   9 | Best Ratio: 0.870 | Best Time Score: 0.499
Gen  10 | Best Ratio: 0.870 | Best Time Score: 0.499
Gen  11 | Best Ratio: 0.870 | Best Time Score: 0.499
Gen  12 | Best Ratio: 0.870 | Best Time Score: 0.499
Gen  13 | Best Ratio: 0.870 | Best Time Score: 0.499


KeyboardInterrupt: 